# Pipeline

## 1. Import libraries

In [2]:
import pandas as pd
import numpy as np
import tqdm
import os
import seaborn as sns
import matplotlib.pyplot as ply
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

## 2. Load dataframes

In [3]:
file_amazon = 'output_01_data_01_AM_mun_especie.csv'
path_input_file_amazon = os.path.join(os.getcwd(), '..', 'sprint_03_data_analysis', 'output', file_amazon)
df_amazon = pd.read_csv(path_input_file_amazon, delimiter=';')
del df_amazon['Unnamed: 0']
df_amazon.head()

,CD,Municipio,Falciparum,Mista,Vivax,Malarie,Ovale,Ano,Estado
0,110001,Alta Floresta D'Oeste,15.000,5.0,117.000,0,0,2003,RO
1,110002,Ariquemes,2.329,138.0,4.801,0,0,2003,RO
2,110003,Cabixi,178.000,0.0,54.000,0,0,2003,RO
3,110004,Cacoal,137.000,9.0,279.000,0,0,2003,RO
4,110005,Cerejeiras,57.000,6.0,104.000,0,0,2003,RO


In [ ]:
file_malarie_covid_total = 'Covid_malaria_total_casos.csv'
path_input_file_malarie = os.path.join(os.getcwd(), '..', 
                                       'sprint_01_data_collection', 
                                       'data_04', 
                                       file_malarie_covid_total)
df_malarie_covid_total = pd.read_csv(path_input_file_malarie)
df_malarie_covid_total.head()

In [4]:
file_covid = 'output_02_data_02_covid_cities_confirmed_cases.csv'
path_input_file_covid = os.path.join(os.getcwd(), '..', 
                                     'sprint_03_data_analysis', 'output', 
                                     file_covid)
df_covid = pd.read_csv(path_input_file_covid, delimiter=';')
del df_covid['Unnamed: 0']
df_covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-05-02,18,22945.0,1.0,1,0,0,NaN,NaN,0.0
1,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-05-03,19,22945.0,1.0,0,0,0,NaN,NaN,0.0
2,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-05-04,19,22945.0,1.0,0,0,0,NaN,NaN,0.0
3,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-05-05,19,22945.0,1.0,0,0,0,NaN,NaN,0.0
4,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-05-06,19,22945.0,1.0,0,0,0,NaN,NaN,0.0


In [ ]:
file_covid_deaths = 'output_03_data_02_covid_cities_confirmed_deaths.csv'
path_input_file_covid_deaths = os.path.join(os.getcwd(), '..', 
                                     'sprint_03_data_analysis', 'output', 
                                     file_covid_deaths)
df_covid_deaths = pd.read_csv(path_input_file_covid_deaths, delimiter=';')
del df_covid_deaths['Unnamed: 0']
df_covid_deaths.head()

### 3.1 df covid: mean of new daily new cases in Brazil

In [ ]:
df_covid['casosNovos'].mean()

### 3.2 df covid: mean of new daily new cases in Brazil by state

In [ ]:
df_covid_mean_newcases_by_state = df_covid[['estado', 'casosNovos']].groupby(['estado']).agg(
    {'casosNovos': ['min', 'mean', 'std', 'max']}).reset_index()

df_covid_mean_newcases_by_state.head()

df_covid_mean_newcases_by_state['casosNovos']['mean']
# df_covid_mean_newcases_by_state['estado']
# df_covid_mean_newcases_by_state.index.tolist()

### 3.3 df covid: mean of new cases in Brazil by city

In [ ]:
df_covid[['codmun', 'casosNovos']].groupby('codmun').agg({'casosNovos': ['min', 'mean', 'std', 'max']}).head()

### 4.1 df covid: (sum of new cases) / length population, in Brazil

In [ ]:
df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
n_pop_BRA = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates()['populacaoTCU2019'].sum()
n_pop_BRA

In [ ]:
df_covid['casosNovos'].sum() / n_pop_BRA

### 4.2 df covid: (sum of new cases) / length population, in Brazil by state

In [ ]:
df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(subset=['codmun']).head()

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(
    subset=['codmun']).groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

In [ ]:
df_covid_sum_newcases_by_state = df_covid[['estado', 'casosNovos']].groupby(['estado']).agg(
    {'casosNovos': ['sum']})

df_covid_sum_newcases_by_state.head()

In [ ]:
df_covid_rate_infected_by_state = df_covid_sum_newcases_by_state['casosNovos'][['sum']] / \
df_covid_sum_population_by_state['populacaoTCU2019'][['sum']]

df_covid_rate_infected_by_state.head()

### 4.3 df covid: (sum of new cases) / length population, in Brazil by city

In [ ]:
df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
df_covid_sum_newcases_by_city = df_covid[['codmun', 'casosNovos']].groupby(['codmun']).agg(
    {'casosNovos': ['sum']})

df_covid_sum_newcases_by_city.head()

In [ ]:
df_covid_rate_infected_by_city = pd.DataFrame(df_covid_sum_newcases_by_city['casosNovos']['sum'] / \
df_covid_sum_population_by_city['populacaoTCU2019'], columns=['infected rate'])

df_covid_rate_infected_by_city.head()

## Since the first case ...

### 5.1 df covid: mean of new deaths in Brazil

In [ ]:
df_covid['obitosNovos'].mean()

### 5.2 df covid: mean of new deaths in Brazil by state

In [ ]:
df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg({'obitosNovos': ['min', 'mean', 'std', 'max']}).head()

### 5.3 df covid: mean of new deaths in Brazil by city

In [ ]:
df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg({'obitosNovos': ['min', 'mean', 'std', 'max']}).head()

## Since the first death ...

### 6.1 df covid deaths: mean of new deaths in Brazil

In [ ]:
df_covid_deaths['obitosNovos'].mean()

### 6.2 df covid deaths: mean of new deaths in Brazil by state

In [ ]:
df_covid_deaths[['estado', 'obitosNovos']].groupby(['estado']).agg({'obitosNovos': ['min', 'mean', 'std', 'max']}).head()

### 6.3 df covid deaths: mean of new deaths in Brazil by city

In [ ]:
df_covid_deaths[['codmun', 'obitosNovos']].groupby('codmun').agg({'obitosNovos': ['min', 'mean', 'std', 'max']}).head()

## Since the first case ...

### 7.1 df covid: (sum of new deaths) / length population, in Brazil

In [ ]:
df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
n_pop_BRA = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates()['populacaoTCU2019'].sum()
n_pop_BRA

In [ ]:
df_covid['obitosNovos'].sum() / n_pop_BRA

### 7.2 df covid: (sum of new deaths) / length population, in Brazil by state

In [ ]:
df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(subset=['codmun']).head()

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(
    subset=['codmun']).groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

In [ ]:
df_covid_sum_newdeaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['sum']})

df_covid_sum_newdeaths_by_state.head()

In [ ]:
df_covid_rate_deaths_by_state = df_covid_sum_newdeaths_by_state['obitosNovos'][['sum']] / \
df_covid_sum_population_by_state['populacaoTCU2019'][['sum']]

df_covid_rate_deaths_by_state.head()

### 7.3 df covid: (sum of new deaths) / length population, in Brazil by city

In [ ]:
df_covid[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
df_covid_sum_newdeaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby(['codmun']).agg(
    {'obitosNovos': ['sum']})

df_covid_sum_newdeaths_by_city.head()

In [ ]:
df_covid_rate_death_by_city = pd.DataFrame(df_covid_sum_newdeaths_by_city['obitosNovos']['sum'] / \
df_covid_sum_population_by_city['populacaoTCU2019'], columns=['death rate'])

df_covid_rate_death_by_city.head()

## since the first death ...

### 8.1 df covid deaths: (sum of new deaths) / length population, in Brazil

In [ ]:
df_covid_deaths[['codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
n_pop_BRA_covid_deaths = df_covid_deaths[['codmun', 'populacaoTCU2019']].drop_duplicates()['populacaoTCU2019'].sum()
n_pop_BRA_covid_deaths

In [ ]:
df_covid_deaths['obitosNovos'].sum() / n_pop_BRA_covid_deaths

### 8.2 df covid deaths: (sum of new deaths) / length population, in Brazil by state

In [ ]:
df_covid_deaths[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(subset=['codmun']).head()

In [ ]:
df_covid_deaths_sum_population_by_state = df_covid_deaths[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates(
    subset=['codmun']).groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_deaths_sum_population_by_state.head()

In [ ]:
df_covid_deaths_sum_newdeaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['sum']})

df_covid_deaths_sum_newdeaths_by_state.head()

In [ ]:
df_covid_deaths_rate_deaths_by_state = df_covid_deaths_sum_newdeaths_by_state['obitosNovos'][['sum']] / \
df_covid_deaths_sum_population_by_state['populacaoTCU2019'][['sum']]

df_covid_deaths_rate_deaths_by_state.head()

### 8.3 df covid deaths: (sum of new deaths) / length population, in Brazil by city

In [ ]:
df_covid_deaths[['estado', 'codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
df_covid_deaths_sum_population_by_city = df_covid_deaths[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_deaths_sum_population_by_city.head()

In [ ]:
df_covid_deaths_sum_newdeaths_by_city = df_covid_deaths[['codmun', 'obitosNovos']].groupby(['codmun']).agg(
    {'obitosNovos': ['sum']})

df_covid_deaths_sum_newdeaths_by_city.head()

In [ ]:
df_covid_deaths_rate_death_by_city = pd.DataFrame(df_covid_deaths_sum_newdeaths_by_city['obitosNovos']['sum'] / \
df_covid_deaths_sum_population_by_city['populacaoTCU2019'], columns=['death rate'])

df_covid_deaths_rate_death_by_city.head()

## Lethality

### 9.1 df covid: (sum of new deaths) / (sum of new cases), in Brazil

In [ ]:
df_covid['obitosNovos'].sum() / df_covid['casosNovos'].sum()

### 9.2 df covid: (sum of new deaths) / (sum of new cases), in Brazil by state

In [ ]:
df_covid_sum_newdeaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['sum']})

df_covid_sum_newdeaths_by_state.head()

In [ ]:
df_covid_sum_newcases_by_state = df_covid[['estado', 'casosNovos']].groupby(['estado']).agg(
    {'casosNovos': ['sum']})

df_covid_sum_newcases_by_state.head()

In [ ]:
df_covid_lethality_by_state = df_covid_sum_newdeaths_by_state['obitosNovos'][['sum']] / \
df_covid_sum_newcases_by_state['casosNovos'][['sum']]

df_covid_lethality_by_state.head()

### 9.3 df covid: (sum of new deaths) / (sum of new cases), in Brazil by city

In [ ]:
df_covid_sum_newdeaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby(['codmun']).agg(
    {'obitosNovos': ['sum']})

df_covid_sum_newdeaths_by_city.head()

In [ ]:
df_covid_sum_newcases_by_city = df_covid[['codmun', 'casosNovos']].groupby(['codmun']).agg(
    {'casosNovos': ['sum']})

df_covid_sum_newcases_by_city.head()

In [ ]:
df_covid_lethality_by_city = df_covid_sum_newdeaths_by_city['obitosNovos'][['sum']] / \
df_covid_sum_newcases_by_city['casosNovos'][['sum']]

df_covid_lethality_by_city.head()

## Covid pressure in Brazil in tottaly, by state, and by city

### 10.1 df covid: df_covid['data'].unique(), in Brazil

In [ ]:
df_covid['data'].unique().shape

### 10.2 df covid: df_covid['data'].unique(), in Brazil by state

In [ ]:
df_covid[['estado', 'codmun', 'data']].groupby(by='estado').head(1)

In [ ]:
"""
https://stackoverflow.com/questions/45759966/counting-unique-values-in-a-column-in-pandas-dataframe-like-in-qlik

 __________________________________________________
 nunique | Count distinct values
 ________|_________________________________________
 count   | Count only non-null values
 ________|_________________________________________
 size    | Count total values including null values
 ________|_________________________________________
 
"""

df_covid[['estado', 'data']].groupby(by='estado').agg(
    {'data': ['nunique', 'count', 'size']}).head()

In [ ]:
df_covid[['estado', 'data']].groupby(by='estado').agg(
    {'data': ['nunique', 'count', 'size']}).sort_values(
    by=('data', 'count'), ascending=False).head()

In [ ]:
"""
Proof of 9
"""
for estate in df_covid[['estado', 'codmun', 'data']]['estado'].unique():
#     print(df_covid[df_covid['estado'] == estate][['estado', 'codmun', 'data']].head())
    df_temp = df_covid[df_covid['estado'] == estate][['data']]
    unique_dates = np.unique(df_temp['data']).shape[0]
    print(estate, unique_dates)
#     break
    del df_temp

print(df_covid[['estado', 'codmun', 'data']][['data']].iloc[0, 0])
print(df_covid[['estado', 'codmun', 'data']][['data']].iloc[1, 0])

### 10.3 df covid: df_covid['data'].unique(), in Brazil by city

In [ ]:
df_covid[['estado', 'codmun', 'data']].groupby(by='estado').head(1)

In [ ]:
"""
https://stackoverflow.com/questions/45759966/counting-unique-values-in-a-column-in-pandas-dataframe-like-in-qlik

 __________________________________________________
 nunique | Count distinct values
 ________|_________________________________________
 count   | Count only non-null values
 ________|_________________________________________
 size    | Count total values including null values
 ________|_________________________________________
 
"""

df_covid[['codmun', 'data']].groupby(by='codmun').agg(
    {'data': ['nunique', 'count', 'size']}).head()

In [ ]:
df_covid[['codmun', 'data']].groupby(by='codmun').agg(
    {'data': ['nunique', 'count', 'size']}).sort_values(
    by=('data', 'count'), ascending=False)

## Mean of deaths by miliion in Brazil in total, by state, and by city

### 11.1 df_covid: (mean of deaths / population) * 10⁹ in Brazil

In [ ]:
df_covid['obitosNovos'].mean()

In [ ]:
df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().head()

In [ ]:
n_pop_BRA = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates()['populacaoTCU2019'].sum()
n_pop_BRA

In [ ]:
(df_covid['obitosNovos'].mean() / n_pop_BRA ) * 10**9

### 11.2 df_covid: (mean of deaths / population) * 10⁹ in Brazil, by estate

In [ ]:
df_covid_mean_deaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_state.head()

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'populacaoTCU2019']].drop_duplicates().groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

In [ ]:
pd.DataFrame((df_covid_mean_deaths_by_state['obitosNovos']['mean'] / \
df_covid_sum_population_by_state['populacaoTCU2019']['sum']) * 10**9, 
             columns=['deaths by million']).head()

### 11.3 df_covid: (mean of deaths / population) * 10⁹ in Brazil, by city

In [ ]:
df_covid_mean_deaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_city.head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
pd.DataFrame((df_covid_mean_deaths_by_city['obitosNovos']['mean'] / \
df_covid_sum_population_by_city['populacaoTCU2019']) * 10**9, 
             columns=['deaths by million']).head()

# <center> Relationship between Covid and Malaria, for all years</center>

## <center>Deaths by million, by state</center>

### 12.1 (df covid: deaths by million) vs (df amazon: num cases p. vivax)

In [ ]:
df_covid_mean_deaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_state.head()

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'populacaoTCU2019']].drop_duplicates().groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

In [ ]:
df_covid_by_state_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_state['obitosNovos']['mean'] / \
df_covid_sum_population_by_state['populacaoTCU2019']['sum']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_state_deaths_by_million.head()

In [ ]:
df_amazon_by_state_sum_vivax = df_amazon.groupby(['Estado']).agg(
    {'Vivax': ['sum']}).drop('Total')

df_amazon_by_state_sum_vivax

In [ ]:
amazon_states = df_amazon.Estado.drop_duplicates().tolist()
amazon_states.remove('Total')
amazon_states

In [ ]:
bol_amazon_states = np.array(
    [state in amazon_states for state in df_covid_by_state_deaths_by_million.index]
).reshape(df_covid_by_state_deaths_by_million.shape[0], 1)

df_covid_by_state_deaths_by_million[bol_amazon_states]

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_state = pd.merge(
    left=df_covid_by_state_deaths_by_million[bol_amazon_states].reset_index(),
    right=df_amazon_by_state_sum_vivax.reset_index().rename(columns={'Estado': 'estado'}),
    on='estado').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_vivax_by_state

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_state[['estado', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_state[['estado', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_state[[('Vivax', 'sum')]].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_state[[('Vivax', 'sum')]].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].mean()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].std()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)]

### 12.2 (df covid: deaths by million) vs (df amazon: num cases p. falciparum)

In [ ]:
df_covid_mean_deaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_state.head()

# ok

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'populacaoTCU2019']].drop_duplicates().groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

# ok

In [ ]:
df_covid_by_state_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_state['obitosNovos']['mean'] / \
df_covid_sum_population_by_state['populacaoTCU2019']['sum']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_state_deaths_by_million.head()

# ok

In [ ]:
df_amazon_by_state_sum_falciparum = df_amazon.groupby(['Estado']).agg(
    {'Falciparum': ['sum']}).drop('Total')

df_amazon_by_state_sum_falciparum

# ok

In [ ]:
amazon_states = df_amazon.Estado.drop_duplicates().tolist()
amazon_states.remove('Total')
amazon_states

In [ ]:
bol_amazon_states = np.array(
    [state in amazon_states for state in df_covid_by_state_deaths_by_million.index]
).reshape(df_covid_by_state_deaths_by_million.shape[0], 1)

df_covid_by_state_deaths_by_million[bol_amazon_states]

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state = pd.merge(
    left=df_covid_by_state_deaths_by_million[bol_amazon_states].reset_index(),
    right=df_amazon_by_state_sum_falciparum.reset_index().rename(columns={'Estado': 'estado'}),
    on='estado').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_falciparum_by_state

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[[('Falciparum', 'sum')]].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[[('Falciparum', 'sum')]].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].mean()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].std()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)]

### 12.3 (df covid: deaths by million) vs (df amazon: num cases p. vivax + p. falciparum)

In [ ]:
df_covid_mean_deaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_state.head()

# ok

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'populacaoTCU2019']].drop_duplicates().groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

# ok

In [ ]:
df_covid_by_state_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_state['obitosNovos']['mean'] / \
df_covid_sum_population_by_state['populacaoTCU2019']['sum']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_state_deaths_by_million.head()

# ok

In [ ]:
df_amazon_by_state_sum_vivax_falciparum = pd.DataFrame(
    df_amazon.groupby(['Estado']).agg(
        {'Vivax': ['sum'], 'Falciparum': ['sum']}).drop('Total').sum(axis=1), 
    columns=['sum_vivax_falciparum'])

df_amazon_by_state_sum_vivax_falciparum

In [ ]:
amazon_states = df_amazon.Estado.drop_duplicates().tolist()
amazon_states.remove('Total')
amazon_states

In [ ]:
bol_amazon_states = np.array(
    [state in amazon_states for state in df_covid_by_state_deaths_by_million.index]
).reshape(df_covid_by_state_deaths_by_million.shape[0], 1)

df_covid_by_state_deaths_by_million[bol_amazon_states]

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_state = pd.merge(
    left=df_covid_by_state_deaths_by_million[bol_amazon_states].reset_index(),
    right=df_amazon_by_state_sum_vivax_falciparum.reset_index().rename(columns={'Estado': 'estado'}),
    on='estado').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_vivax_falciparum_by_state

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_state[['sum_vivax_falciparum']].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_state[[('sum_vivax_falciparum')]].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].mean()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].std()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)]

### 12.4 (df covid: deaths by million) vs (df amazon: num cases all p. species)

In [ ]:
df_covid_mean_deaths_by_state = df_covid[['estado', 'obitosNovos']].groupby(['estado']).agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_state.head()

In [ ]:
df_covid_sum_population_by_state = df_covid[['estado', 'populacaoTCU2019']].drop_duplicates().groupby(by=['estado']
).agg({
    'populacaoTCU2019': [sum]
})

df_covid_sum_population_by_state.head()

In [ ]:
df_covid_by_state_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_state['obitosNovos']['mean'] / \
df_covid_sum_population_by_state['populacaoTCU2019']['sum']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_state_deaths_by_million.head()

In [ ]:
df_amazon_by_state_sum_all_plasmodium_species = pd.DataFrame(
    df_amazon.groupby(['Estado']).agg(
        {'Vivax': ['sum'], 
         'Falciparum': ['sum'], 
         'Malarie': ['sum'], 
         'Ovale': ['sum'], 
         'Mista': ['sum']}).drop('Total').sum(axis=1), 
    columns=['sum_all_plasmodium_species'])

df_amazon_by_state_sum_all_plasmodium_species

In [ ]:
amazon_states = df_amazon.Estado.drop_duplicates().tolist()
amazon_states.remove('Total')
amazon_states

In [ ]:
bol_amazon_states = np.array(
    [state in amazon_states for state in df_covid_by_state_deaths_by_million.index]
).reshape(df_covid_by_state_deaths_by_million.shape[0], 1)

df_covid_by_state_deaths_by_million[bol_amazon_states]

In [ ]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_state = pd.merge(
    left=df_covid_by_state_deaths_by_million[bol_amazon_states].reset_index(),
    right=df_amazon_by_state_sum_all_plasmodium_species.reset_index().rename(columns={'Estado': 'estado'}),
    on='estado').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_all_plasmodium_species_by_state

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_state[['estado', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_state[['sum_all_plasmodium_species']].mean()

In [ ]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_state[['sum_all_plasmodium_species']].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].mean()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)].std()

In [ ]:
df_covid_by_state_deaths_by_million[np.logical_not(bol_amazon_states)]

## <center>Deaths by million, by city</center>

### 13.1 (df covid: deaths by million) vs (df amazon: num cases p. vivax)

In [ ]:
df_covid_mean_deaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_city.head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
df_covid_by_city_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_city['obitosNovos']['mean'] / \
df_covid_sum_population_by_city['populacaoTCU2019']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_city_deaths_by_million.index = df_covid_by_city_deaths_by_million.index.map(str).map(lambda codmun: codmun.split('.')[0])

df_covid_by_city_deaths_by_million.head()

In [ ]:
df_amazon_by_city_sum_vivax = df_amazon.groupby(['CD']).agg(
    {'Vivax': ['sum']}).drop('Total geral')

df_amazon_by_city_sum_vivax.head()

In [ ]:
amazon_cities = df_amazon.CD.drop_duplicates().tolist()
amazon_cities.remove('Total geral')

print(len(amazon_cities))
amazon_cities[:10]

In [ ]:
bol_amazon_cities = np.array(
    [city in amazon_cities for city in df_covid_by_city_deaths_by_million.index]
).reshape(df_covid_by_city_deaths_by_million.shape[0], 1)

df_covid_by_city_deaths_by_million[bol_amazon_cities]

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_city = pd.merge(
    left=df_covid_by_city_deaths_by_million[bol_amazon_cities].reset_index(),
    right=df_amazon_by_city_sum_vivax.reset_index().rename(columns={'CD': 'codmun'}),
    on='codmun').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_vivax_by_city

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_city[['codmun', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_city[['codmun', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_city[[('Vivax', 'sum')]].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_by_city[[('Vivax', 'sum')]].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].mean()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].std()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)]

### 13.2 (df covid: deaths by million) vs (df amazon: num cases p. falciparum)

In [ ]:
df_covid_mean_deaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_city.head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
df_covid_by_city_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_city['obitosNovos']['mean'] / \
df_covid_sum_population_by_city['populacaoTCU2019']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_city_deaths_by_million.index = df_covid_by_city_deaths_by_million.index.map(str).map(lambda codmun: codmun.split('.')[0])

df_covid_by_city_deaths_by_million.head()

In [ ]:
df_amazon_by_city_sum_falciparum = df_amazon.groupby(['CD']).agg(
    {'Falciparum': ['sum']}).drop('Total geral')

df_amazon_by_city_sum_falciparum.head()

In [ ]:
amazon_cities = df_amazon.CD.drop_duplicates().tolist()
amazon_cities.remove('Total geral')

print(len(amazon_cities))
amazon_cities[:10]

In [ ]:
bol_amazon_cities = np.array(
    [city in amazon_cities for city in df_covid_by_city_deaths_by_million.index]
).reshape(df_covid_by_city_deaths_by_million.shape[0], 1)

df_covid_by_city_deaths_by_million[bol_amazon_cities]

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_city = pd.merge(
    left=df_covid_by_city_deaths_by_million[bol_amazon_cities].reset_index(),
    right=df_amazon_by_city_sum_falciparum.reset_index().rename(columns={'CD': 'codmun'}),
    on='codmun').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_falciparum_by_city

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_city[['codmun', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_city[['codmun', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_city[[('Falciparum', 'sum')]].mean()

In [ ]:
df_deaths_by_million_vs_sum_falciparum_by_city[[('Falciparum', 'sum')]].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].mean()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].std()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)]

### 13.3 (df covid: deaths by million) vs (df amazon: num cases p. vivax + p. falciparum)

In [ ]:
df_covid_mean_deaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_city.head()

In [ ]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [ ]:
df_covid_by_city_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_city['obitosNovos']['mean'] / \
df_covid_sum_population_by_city['populacaoTCU2019']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_city_deaths_by_million.index = df_covid_by_city_deaths_by_million.index.map(str).map(lambda codmun: codmun.split('.')[0])

df_covid_by_city_deaths_by_million.head()

In [ ]:
df_amazon_by_city_sum_vivax_falciparum = pd.DataFrame(
    df_amazon.groupby(['CD']).agg(
        {'Vivax': ['sum'], 
         'Falciparum': ['sum']}).drop('Total geral').sum(axis=1), 
    columns=['sum_vivax_falciparum'])

df_amazon_by_city_sum_vivax_falciparum.head()

In [ ]:
amazon_cities = df_amazon.CD.drop_duplicates().tolist()
amazon_cities.remove('Total geral')

print(len(amazon_cities))
amazon_cities[:10]

In [ ]:
bol_amazon_cities = np.array(
    [city in amazon_cities for city in df_covid_by_city_deaths_by_million.index]
).reshape(df_covid_by_city_deaths_by_million.shape[0], 1)

df_covid_by_city_deaths_by_million[bol_amazon_cities]

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_city = pd.merge(
    left=df_covid_by_city_deaths_by_million[bol_amazon_cities].reset_index(),
    right=df_amazon_by_city_sum_vivax_falciparum.reset_index().rename(columns={'CD': 'codmun'}),
    on='codmun').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_vivax_falciparum_by_city

#### <center> Amazon </center>

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_city[['codmun', 'deaths by million']].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_city[['codmun', 'deaths by million']].std()

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_city[['sum_vivax_falciparum']].mean()

In [ ]:
df_deaths_by_million_vs_sum_vivax_falciparum_by_city[['sum_vivax_falciparum']].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].mean()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].std()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)]

### 13.4 (df covid: deaths by million) vs (df amazon: num cases all p. species)

In [13]:
df_covid_mean_deaths_by_city = df_covid[['codmun', 'obitosNovos']].groupby('codmun').agg(
    {'obitosNovos': ['mean']})

df_covid_mean_deaths_by_city.head()

In [15]:
df_covid_sum_population_by_city = df_covid[['codmun', 'populacaoTCU2019']].drop_duplicates().groupby(
    by='codmun').agg(
    {'populacaoTCU2019': 'sum'})

df_covid_sum_population_by_city.head()

In [18]:
df_covid_by_city_deaths_by_million = pd.DataFrame((df_covid_mean_deaths_by_city['obitosNovos']['mean'] / \
df_covid_sum_population_by_city['populacaoTCU2019']) * 10**9, 
             columns=['deaths by million'])

df_covid_by_city_deaths_by_million.index = df_covid_by_city_deaths_by_million.index.map(str).map(lambda codmun: codmun.split('.')[0])

df_covid_by_city_deaths_by_million.head()

In [26]:
df_amazon_by_city_sum_all_plasmodium_species = pd.DataFrame(
    df_amazon.groupby(['CD']).agg(
        {'Vivax': ['sum'], 
         'Falciparum': ['sum'], 
         'Malarie': ['sum'], 
         'Ovale': ['sum'], 
         'Mista': ['sum']}).drop('Total geral').sum(axis=1), 
    columns=['sum_all_plasmodium_species'])

df_amazon_by_city_sum_all_plasmodium_species.head()

In [20]:
amazon_cities = df_amazon.CD.drop_duplicates().tolist()
amazon_cities.remove('Total geral')

print(len(amazon_cities))
amazon_cities[:10]

In [22]:
bol_amazon_cities = np.array(
    [city in amazon_cities for city in df_covid_by_city_deaths_by_million.index]
).reshape(df_covid_by_city_deaths_by_million.shape[0], 1)

df_covid_by_city_deaths_by_million[bol_amazon_cities]

In [28]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_city = pd.merge(
    left=df_covid_by_city_deaths_by_million[bol_amazon_cities].reset_index(),
    right=df_amazon_by_city_sum_all_plasmodium_species.reset_index().rename(columns={'CD': 'codmun'}),
    on='codmun').sort_values(by=('deaths by million'), ascending=False)

df_deaths_by_million_vs_sum_all_plasmodium_species_by_city

#### <center> Amazon </center>

In [33]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_city[['codmun', 'deaths by million']].mean()

In [34]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_city[['codmun', 'deaths by million']].std()

In [35]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_city[['sum_all_plasmodium_species']].mean()

In [36]:
df_deaths_by_million_vs_sum_all_plasmodium_species_by_city[['sum_all_plasmodium_species']].std()

#### <center>No Amazon</center>

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].mean()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)].std()

In [ ]:
df_covid_by_city_deaths_by_million[np.logical_not(bol_amazon_cities)]